In [1]:
from dotenv import load_dotenv

import os

load_dotenv()

True

In [2]:
GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY ")

UPSTASH_VECTOR_REST_TOKEN = os.environ.get("UPSTASH_VECTOR_REST_TOKEN ")

UPSTASH_VECTOR_REST_URL = os.environ.get("UPSTASH_VECTOR_REST_URL")

In [3]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [4]:
from langchain_community.vectorstores import UpstashVectorStore


store = UpstashVectorStore(
    embedding = embeddings,
    index_url = UPSTASH_VECTOR_REST_URL ,
    index_token = UPSTASH_VECTOR_REST_TOKEN
)

In [5]:
import pathlib
from langchain_core.documents import Document
from PyPDF2 import PdfReader


BASE_DIR = pathlib.Path("__file__").parent.resolve()

DATASET_DIR = BASE_DIR/"dataset"

documents = []
# for i in DATASET_DIR.iterdir():
#     reader = PdfReader(i)
 
#     for page in reader.pages:
#         text = page.extract_text()
#         texts.append(text)

katzung = DATASET_DIR/"Katzung Basic and Clinical Pharmacology - 13E (2015) [TruePDF] [UnitedVRG] c.pdf"
tripathi = DATASET_DIR/"KD-Tripathi-Pharmacology-Book.pdf"
lippincot = DATASET_DIR/"Lippincot Pharmacology 5th edition.pdf"

textbooks = [katzung, tripathi, lippincot]

# for textbook in textbooks:
#     reader = PdfReader(str(textbook))
     
#     for page in reader.pages:
#         text = page.extract_text()
    
#         doc = Document(
#             page_content = text,
#             metadata = {
#                 "source": os.path.basename(textbook),
#                 "title": os.path.basename(textbook)
#             }
#         )
#         documents.append(doc)

                


In [6]:
# documents[2].metadata

In [7]:
# from langchain_text_splitters import TokenTextSplitter

# text_splitter = TokenTextSplitter.from_tiktoken_encoder(model_name = "gpt-4o", chunk_size=100, chunk_overlap=0)

In [8]:
# docs = text_splitter.split_documents(documents)

In [9]:
# insert_vectors = store.add_documents(docs[10000:])

In [11]:
# len(docs)

In [14]:
result = store.similarity_search_with_score("list antihypertensive drugs", k=2)

for doc, score in result:
    print(f"{doc.page_content} - {score}")

ENTS
All antihypertensive agents act at one or more of the four ana-
tomic control sites depicted in Figure 11–1 and produce their effects by interfering with normal mechanisms of blood pressure regulation. A useful classification of these agents categorizes them according to the principal regulatory site or mechanism on which they act (Figure 11–3). Because of their common mechanisms of action, drugs within each category tend to produce a similar spec-trum of toxicities. The categories include the following - 0.8766016
 of drugs includes the oral vasodilators, hydralazine and 
minoxidil, which are used for long-term outpatient therapy of 
hypertension; the parenteral vasodilators, nitroprusside, diazoxide, 
and fenoldopam, which are used to treat hypertensive emergencies; 
the calcium channel blockers, which are used in both circumstances; 
and the nitrates, which are used mainly in angina (Table 11–3).
Chapter  - 0.87549305


In [35]:
retriever = store.as_retriever(
    search_type = "similarity",
    search_kwargs = {'k':4}
)

In [36]:
from langchain_google_genai import GoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser

LLM_CONFIG = {
    "model": "gemini-1.5-pro",
    "google_api_key": GOOGLE_API_KEY

}

llm = GoogleGenerativeAI(**LLM_CONFIG)

In [37]:
from langchain_core.prompts import ChatPromptTemplate

message = '''
Answer this question using the provided context only.

{question}

Context:
{context}
'''

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "if the user is greeting then respond to the greeting"),
        ("human", message)
    ]
)

In [38]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed = RunnablePassthrough(),
    modified = lambda x: x["num"] + 1
)

parser = StrOutputParser()

In [39]:
chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm | parser

In [43]:
chain.invoke("How are you?")

"I'm doing well, thank you for asking. How can I help you today?\n"